In [1]:
import random

import numpy as np
import torch
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
data = np.load(r'D:\heat_wave\pacific\last\wind_93_19_pacific_area.npz')
print(data.files)  # ['time', 'lat', 'lon', 'eastward_wind', 'northward_wind', 'wind_vector_divergence', 'wind_stress', 'surface_downward_eastward_stress', 'surface_downward_northward_stress', 'wind_stress_curl', 'wind_stress_divergence', 'wind_speed_rms', 'eastward_wind_rms', 'northward_wind_rms', 'sampling_length', 'surface_type']

eastward_wind = data['eastward_wind'][:, :40, :200]  # (9861, 41, 201)  #
print(eastward_wind.shape)  # (9861, 41, 201)
northward_wind = data['northward_wind'][:, :40, :200]  # (9861, 41, 201)
wind_vector_divergence = data['wind_vector_divergence'][:, :40, :200]  # (9861, 41, 201)
wind_stress = data['wind_stress'][:, :40, :200]  # (9861, 41, 201)   # 可惜这个是 0
surface_downward_eastward_stress = data['surface_downward_eastward_stress'][:, :40, :200]  # (9861, 41, 201)
surface_downward_northward_stress = data['surface_downward_northward_stress'][:, :40, :200]  # (9861, 41, 201)
wind_stress_curl = data['wind_stress_curl'][:, :40, :200]  # (9861, 41, 201)
wind_stress_divergence = data['wind_stress_divergence'][:, :40, :200]  # (9861, 41, 201)
wind_speed_rms = data['wind_speed_rms'][:, :40, :200]  # (9861, 41, 201)
eastward_wind_rms = data['eastward_wind_rms'][:, :40, :200]  # (9861, 41, 201)
northward_wind_rms = data['northward_wind_rms'][:, :40, :200]  # (9861, 41, 201)
sampling_length = data['sampling_length'][:, :40, :200]  # (9861, 41, 201)
surface_type = data['surface_type'][:, :40, :200]  # (9861, 41, 201)

['time', 'lat', 'lon', 'eastward_wind', 'northward_wind', 'wind_vector_divergence', 'wind_stress', 'surface_downward_eastward_stress', 'surface_downward_northward_stress', 'wind_stress_curl', 'wind_stress_divergence', 'wind_speed_rms', 'eastward_wind_rms', 'northward_wind_rms', 'sampling_length', 'surface_type']
(9848, 40, 200)


In [ ]:
# 首先 我们先画 SON季节的贡献图   JJA： 9  10  11 月  即  09/01 - 11/31  当然，会做一个区域平均
# 同时，我们也会对每一年的JJA做一个平均  ---》1993 - 2019年

In [ ]:
#  计算索引
import pandas as pd
import datetime

#创建时间范围
start_date = datetime.datetime(1993,1,1)
end_date = datetime.datetime(2019,12,31)
dates = pd.date_range(start = start_date, end = end_date, freq = 'D')

index = dates.get_loc(datetime.datetime(2019,11,30)) 
index

In [ ]:
# SON   93
time_93_1 = 243
time_93_2 = 334
data_1 = time_93_1
data_2 = time_93_2

eastward_wind_SON_93 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_93 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_93 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_93 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_93 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_93 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_93 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_93 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_93 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_93 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_93 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_93 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_93 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   94
time_94_1 = 608
time_94_2 = 699
data_1 = time_94_1
data_2 = time_94_2

eastward_wind_SON_94 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_94 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_94 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_94 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_94 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_94 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_94 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_94 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_94 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_94 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_94 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_94 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_94 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   95
time_95_1 = 973
time_95_2 = 1064
data_1 = time_95_1
data_2 = time_95_2

eastward_wind_SON_95 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_95 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_95 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_95 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_95 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_95 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_95 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_95 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_95 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_95 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_95 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_95 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_95 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   96
time_96_1 = 1339
time_96_2 = 1430
data_1 = time_96_1
data_2 = time_96_2

eastward_wind_SON_96 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_96 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_96 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_96 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_96 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_96 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_96 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_96 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_96 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_96 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_96 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_96 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_96 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   97
time_97_1 = 1704
time_97_2 = 1795
data_1 = time_97_1
data_2 = time_97_2

eastward_wind_SON_97 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_97 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_97 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_97 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_97 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_97 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_97 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_97 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_97 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_97 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_97 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_97 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_97 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   98
time_98_1 = 2069
time_98_2 = 2160
data_1 = time_98_1
data_2 = time_98_2

eastward_wind_SON_98 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_98 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_98 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_98 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_98 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_98 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_98 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_98 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_98 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_98 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_98 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_98 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_98 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   99
time_99_1 = 2434
time_99_2 = 2525
data_1 = time_99_1
data_2 = time_99_2

eastward_wind_SON_99 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_99 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_99 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_99 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_99 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_99 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_99 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_99 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_99 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_99 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_99 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_99 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_99 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   00
time_00_1 = 2800
time_00_2 = 2891
data_1 = time_00_1
data_2 = time_00_2

eastward_wind_SON_00 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_00 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_00 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_00 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_00 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_00 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_00 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_00 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_00 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_00 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_00 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_00 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_00 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   01
time_01_1 = 3165
time_01_2 = 3256
data_1 = time_01_1
data_2 = time_01_2

eastward_wind_SON_01 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_01 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_01 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_01 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_01 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_01 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_01 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_01 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_01 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_01 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_01 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_01 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_01 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   02
time_02_1 = 3529
time_02_2 = 3620
data_1 = time_02_1
data_2 = time_02_2

eastward_wind_SON_02 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_02 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_02 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_02 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_02 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_02 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_02 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_02 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_02 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_02 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_02 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_02 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_02 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   03
time_03_1 = 3895
time_03_2 = 3986
data_1 = time_03_1
data_2 = time_03_2

eastward_wind_SON_03 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_03 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_03 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_03 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_03 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_03 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_03 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_03 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_03 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_03 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_03 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_03 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_03 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   04
time_04_1 = 4261
time_04_2 = 4352
data_1 = time_04_1
data_2 = time_04_2

eastward_wind_SON_04 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_04 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_04 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_04 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_04 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_04 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_04 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_04 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_04 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_04 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_04 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_04 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_04 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   05
time_05_1 = 4626
time_05_2 = 4717
data_1 = time_05_1
data_2 = time_05_2

eastward_wind_SON_05 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_05 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_05 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_05 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_05 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_05 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_05 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_05 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_05 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_05 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_05 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_05 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_05 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   06
time_06_1 = 4990
time_06_2 = 5081
data_1 = time_06_1
data_2 = time_06_2

eastward_wind_SON_06 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_06 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_06 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_06 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_06 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_06 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_06 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_06 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_06 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_06 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_06 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_06 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_06 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   07
time_07_1 = 5356
time_07_2 = 5447
data_1 = time_07_1
data_2 = time_07_2

eastward_wind_SON_07 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_07 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_07 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_07 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_07 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_07 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_07 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_07 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_07 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_07 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_07 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_07 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_07 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   08
time_08_1 = 5722
time_08_2 = 5813
data_1 = time_08_1
data_2 = time_08_2

eastward_wind_SON_08 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_08 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_08 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_08 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_08 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_08 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_08 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_08 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_08 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_08 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_08 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_08 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_08 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   09
time_09_1 = 6087
time_09_2 = 6178
data_1 = time_09_1
data_2 = time_09_2

eastward_wind_SON_09 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_09 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_09 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_09 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_09 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_09 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_09 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_09 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_09 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_09 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_09 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_09 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_09 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   10
time_10_1 = 6452
time_10_2 = 6543
data_1 = time_10_1
data_2 = time_10_2

eastward_wind_SON_10 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_10 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_10 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_10 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_10 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_10 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_10 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_10 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_10 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_10 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_10 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_10 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_10 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   11
time_11_1 = 6817
time_11_2 = 6908
data_1 = time_11_1
data_2 = time_11_2

eastward_wind_SON_11 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_11 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_11 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_11 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_11 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_11 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_11 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_11 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_11 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_11 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_11 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_11 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_11 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   12
time_12_1 = 7183
time_12_2 = 7274
data_1 = time_12_1
data_2 = time_12_2

eastward_wind_SON_12 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_12 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_12 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_12 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_12 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_12 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_12 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_12 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_12 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_12 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_12 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_12 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_12 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   13
time_13_1 = 7548
time_13_2 = 7639
data_1 = time_13_1
data_2 = time_13_2

eastward_wind_SON_13 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_13 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_13 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_13 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_13 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_13 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_13 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_13 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_13 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_13 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_13 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_13 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_13 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   14
time_14_1 = 7913
time_14_2 = 8004
data_1 = time_14_1
data_2 = time_14_2

eastward_wind_SON_14 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_14 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_14 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_14 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_14 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_14 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_14 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_14 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_14 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_14 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_14 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_14 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_14 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   15
time_15_1 = 8278
time_15_2 = 8369
data_1 = time_15_1
data_2 = time_15_2

eastward_wind_SON_15 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_15 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_15 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_15 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_15 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_15 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_15 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_15 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_15 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_15 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_15 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_15 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_15 = surface_type[data_1 : data_2,:,:]

In [ ]:
# SON   16
time_16_1 = 8644
time_16_2 = 8735
data_1 = time_16_1
data_2 = time_16_2

eastward_wind_SON_16 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_16 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_16 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_16 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_16 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_16 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_16 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_16 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_16 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_16 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_16 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_16 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_16 = surface_type[data_1 : data_2,:,:]

In [29]:
# SON   17
time_17_1 = 9009
time_17_2 = 9100
data_1 = time_17_1
data_2 = time_17_2

eastward_wind_SON_17 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_17 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_17 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_17 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_17 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_17 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_17 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_17 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_17 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_17 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_17 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_17 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_17 = surface_type[data_1 : data_2,:,:]

In [30]:
# SON   18
time_18_1 = 9374
time_18_2 = 9452
data_1 = time_18_1
data_2 = time_18_2

eastward_wind_SON_18 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_18 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_18 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_18 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_18 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_18 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_18 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_18 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_18 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_18 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_18 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_18 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_18 = surface_type[data_1 : data_2,:,:]

In [31]:
# SON   19
time_19_1 = 9739
time_19_2 = 9817
data_1 = time_19_1
data_2 = time_19_2

eastward_wind_SON_19 = eastward_wind[data_1 : data_2,:,:]
northward_wind_SON_19 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_SON_19 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_SON_19 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_SON_19 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_SON_19 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_SON_19 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_SON_19 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_SON_19 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_SON_19 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_SON_19 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_SON_19 = sampling_length[data_1 : data_2,:,:]
surface_type_SON_19 = surface_type[data_1 : data_2,:,:]

In [32]:
wind_stress_SON = np.concatenate((wind_stress_SON_93, wind_stress_SON_94, wind_stress_SON_95,
                                 wind_stress_SON_96, wind_stress_SON_97, wind_stress_SON_98, wind_stress_SON_99,
                                 wind_stress_SON_00, wind_stress_SON_01, wind_stress_SON_02, wind_stress_SON_03,
                                 wind_stress_SON_04, wind_stress_SON_05, wind_stress_SON_06, wind_stress_SON_07,
                                 wind_stress_SON_08, wind_stress_SON_09, wind_stress_SON_10, wind_stress_SON_11,
                                 wind_stress_SON_12, wind_stress_SON_13, wind_stress_SON_14, wind_stress_SON_15,
                                 wind_stress_SON_16, wind_stress_SON_17, wind_stress_SON_18, wind_stress_SON_19), axis = 0)

In [33]:
surface_type_SON = np.concatenate((surface_type_SON_93, surface_type_SON_94, surface_type_SON_95,
                                  surface_type_SON_96, surface_type_SON_97, surface_type_SON_98, surface_type_SON_99,
                                  surface_type_SON_00, surface_type_SON_01, surface_type_SON_02, surface_type_SON_03,
                                  surface_type_SON_04, surface_type_SON_05, surface_type_SON_06, surface_type_SON_07,
                                  surface_type_SON_08, surface_type_SON_09, surface_type_SON_10, surface_type_SON_11,
                                  surface_type_SON_12, surface_type_SON_13, surface_type_SON_14, surface_type_SON_15,
                                  surface_type_SON_16, surface_type_SON_17, surface_type_SON_18, surface_type_SON_19), axis = 0)

In [34]:
eastward_wind_SON = np.concatenate((eastward_wind_SON_93, eastward_wind_SON_94, eastward_wind_SON_95,
                                   eastward_wind_SON_96, eastward_wind_SON_97, eastward_wind_SON_98,
                                   eastward_wind_SON_99, eastward_wind_SON_00, eastward_wind_SON_01,
                                   eastward_wind_SON_02, eastward_wind_SON_03, eastward_wind_SON_04,
                                   eastward_wind_SON_05, eastward_wind_SON_06, eastward_wind_SON_07,
                                   eastward_wind_SON_08, eastward_wind_SON_09, eastward_wind_SON_10,
                                   eastward_wind_SON_11, eastward_wind_SON_12, eastward_wind_SON_13,
                                   eastward_wind_SON_14, eastward_wind_SON_15, eastward_wind_SON_16,
                                   eastward_wind_SON_17, eastward_wind_SON_18, eastward_wind_SON_19), axis = 0)

In [35]:
wind_stress_curl_SON = np.concatenate((wind_stress_curl_SON_93, wind_stress_curl_SON_94, wind_stress_curl_SON_95,
                                      wind_stress_curl_SON_96, wind_stress_curl_SON_97, wind_stress_curl_SON_98, 
                                      wind_stress_curl_SON_99, wind_stress_curl_SON_00, wind_stress_curl_SON_01,
                                      wind_stress_curl_SON_02, wind_stress_curl_SON_03, wind_stress_curl_SON_04, 
                                      wind_stress_curl_SON_05, wind_stress_curl_SON_06, wind_stress_curl_SON_07,
                                      wind_stress_curl_SON_08, wind_stress_curl_SON_09, wind_stress_curl_SON_10,
                                      wind_stress_curl_SON_11, wind_stress_curl_SON_12, wind_stress_curl_SON_13,
                                      wind_stress_curl_SON_14, wind_stress_curl_SON_15, wind_stress_curl_SON_16,
                                      wind_stress_curl_SON_17, wind_stress_curl_SON_18, wind_stress_curl_SON_19), axis = 0)

In [36]:
northward_wind_SON = np.concatenate((northward_wind_SON_93, northward_wind_SON_94, northward_wind_SON_95,
                                    northward_wind_SON_96, northward_wind_SON_97, northward_wind_SON_98,
                                    northward_wind_SON_99, northward_wind_SON_00, northward_wind_SON_01,
                                    northward_wind_SON_02, northward_wind_SON_03, northward_wind_SON_04, 
                                    northward_wind_SON_05, northward_wind_SON_06, northward_wind_SON_07,
                                    northward_wind_SON_08, northward_wind_SON_09, northward_wind_SON_10,
                                    northward_wind_SON_11, northward_wind_SON_12, northward_wind_SON_13,
                                    northward_wind_SON_14, northward_wind_SON_15, northward_wind_SON_16,
                                    northward_wind_SON_17, northward_wind_SON_18, northward_wind_SON_19), axis = 0)

In [37]:
wind_speed_rms_SON = np.concatenate((wind_speed_rms_SON_93, wind_speed_rms_SON_94, wind_speed_rms_SON_95,
                                    wind_speed_rms_SON_96, wind_speed_rms_SON_97, wind_speed_rms_SON_98,
                                    wind_speed_rms_SON_99, wind_speed_rms_SON_00, wind_speed_rms_SON_01,
                                    wind_speed_rms_SON_02, wind_speed_rms_SON_03, wind_speed_rms_SON_04,
                                    wind_speed_rms_SON_05, wind_speed_rms_SON_06, wind_speed_rms_SON_07,
                                    wind_speed_rms_SON_08, wind_speed_rms_SON_09, wind_speed_rms_SON_10,
                                    wind_speed_rms_SON_11, wind_speed_rms_SON_12, wind_speed_rms_SON_13,
                                    wind_speed_rms_SON_14, wind_speed_rms_SON_15, wind_speed_rms_SON_16,
                                    wind_speed_rms_SON_17, wind_speed_rms_SON_18, wind_speed_rms_SON_19), axis = 0)

In [38]:
sampling_length_SON = np.concatenate((sampling_length_SON_93, sampling_length_SON_94, sampling_length_SON_95,
                                     sampling_length_SON_96, sampling_length_SON_97, sampling_length_SON_98,
                                     sampling_length_SON_99, sampling_length_SON_00, sampling_length_SON_01,
                                     sampling_length_SON_02, sampling_length_SON_03, sampling_length_SON_04,
                                     sampling_length_SON_05, sampling_length_SON_06, sampling_length_SON_07,
                                     sampling_length_SON_08, sampling_length_SON_09, sampling_length_SON_10,
                                     sampling_length_SON_11, sampling_length_SON_12, sampling_length_SON_13,
                                     sampling_length_SON_14, sampling_length_SON_15, sampling_length_SON_16,
                                     sampling_length_SON_17, sampling_length_SON_18, sampling_length_SON_19), axis = 0)

In [39]:
eastward_wind_rms_SON = np.concatenate((eastward_wind_rms_SON_93, eastward_wind_rms_SON_94, eastward_wind_rms_SON_95,
                                       eastward_wind_rms_SON_96, eastward_wind_rms_SON_97, eastward_wind_rms_SON_98,
                                       eastward_wind_rms_SON_99, eastward_wind_rms_SON_00, eastward_wind_rms_SON_01,
                                       eastward_wind_rms_SON_02, eastward_wind_rms_SON_03, eastward_wind_rms_SON_04,
                                       eastward_wind_rms_SON_05, eastward_wind_rms_SON_06, eastward_wind_rms_SON_07,
                                       eastward_wind_rms_SON_08, eastward_wind_rms_SON_09, eastward_wind_rms_SON_10,
                                       eastward_wind_rms_SON_11, eastward_wind_rms_SON_12, eastward_wind_rms_SON_13,
                                       eastward_wind_rms_SON_14, eastward_wind_rms_SON_15, eastward_wind_rms_SON_16,
                                       eastward_wind_rms_SON_17, eastward_wind_rms_SON_18, eastward_wind_rms_SON_19), axis = 0)

In [40]:
northward_wind_rms_SON = np.concatenate((northward_wind_rms_SON_93, northward_wind_rms_SON_94,
                                        northward_wind_rms_SON_95, northward_wind_rms_SON_96,northward_wind_rms_SON_97,
                                        northward_wind_rms_SON_98, northward_wind_rms_SON_99, northward_wind_rms_SON_00,
                                        northward_wind_rms_SON_01, northward_wind_rms_SON_02, northward_wind_rms_SON_03,
                                        northward_wind_rms_SON_04, northward_wind_rms_SON_05, northward_wind_rms_SON_06,
                                        northward_wind_rms_SON_07, northward_wind_rms_SON_08, northward_wind_rms_SON_09,
                                        northward_wind_rms_SON_10, northward_wind_rms_SON_11, northward_wind_rms_SON_12,
                                        northward_wind_rms_SON_13, northward_wind_rms_SON_14, northward_wind_rms_SON_15,
                                        northward_wind_rms_SON_16, northward_wind_rms_SON_17, northward_wind_rms_SON_18,
                                        northward_wind_rms_SON_19), axis = 0)

In [41]:
wind_stress_divergence_SON = np.concatenate((wind_stress_divergence_SON_93, wind_stress_divergence_SON_94,
                                            wind_stress_divergence_SON_95, wind_stress_divergence_SON_96, 
                                            wind_stress_divergence_SON_97, wind_stress_divergence_SON_98,
                                            wind_stress_divergence_SON_99, wind_stress_divergence_SON_00,
                                            wind_stress_divergence_SON_01, wind_stress_divergence_SON_02,
                                            wind_stress_divergence_SON_03, wind_stress_divergence_SON_04,
                                            wind_stress_divergence_SON_05, wind_stress_divergence_SON_06, 
                                            wind_stress_divergence_SON_07, wind_stress_divergence_SON_08,
                                            wind_stress_divergence_SON_09, wind_stress_divergence_SON_10,
                                            wind_stress_divergence_SON_11, wind_stress_divergence_SON_12,
                                            wind_stress_divergence_SON_13, wind_stress_divergence_SON_14,
                                            wind_stress_divergence_SON_15, wind_stress_divergence_SON_16,
                                            wind_stress_divergence_SON_17, wind_stress_divergence_SON_18, wind_stress_divergence_SON_19), axis = 0)

In [42]:
wind_vector_divergence_SON = np.concatenate((wind_vector_divergence_SON_93, wind_vector_divergence_SON_94,
                                            wind_vector_divergence_SON_95, wind_vector_divergence_SON_96,
                                            wind_vector_divergence_SON_97, wind_vector_divergence_SON_98,
                                            wind_vector_divergence_SON_99, wind_vector_divergence_SON_00,
                                            wind_vector_divergence_SON_01, wind_vector_divergence_SON_02,
                                            wind_vector_divergence_SON_03, wind_vector_divergence_SON_04,
                                            wind_vector_divergence_SON_05, wind_vector_divergence_SON_06,
                                            wind_vector_divergence_SON_07, wind_vector_divergence_SON_08,
                                            wind_vector_divergence_SON_09, wind_vector_divergence_SON_10,
                                            wind_vector_divergence_SON_11, wind_vector_divergence_SON_12,
                                            wind_vector_divergence_SON_13, wind_vector_divergence_SON_14,
                                            wind_vector_divergence_SON_15, wind_vector_divergence_SON_16,
                                            wind_vector_divergence_SON_17, wind_vector_divergence_SON_18, wind_vector_divergence_SON_19), axis = 0)

In [43]:
surface_downward_eastward_stress_SON = np.concatenate((surface_downward_eastward_stress_SON_93,
                                                      surface_downward_eastward_stress_SON_94, surface_downward_eastward_stress_SON_95,
                                                      surface_downward_eastward_stress_SON_96, surface_downward_eastward_stress_SON_97,
                                                      surface_downward_eastward_stress_SON_98, surface_downward_eastward_stress_SON_99,
                                                      surface_downward_eastward_stress_SON_00, surface_downward_eastward_stress_SON_01,
                                                      surface_downward_eastward_stress_SON_02, surface_downward_eastward_stress_SON_03,
                                                      surface_downward_eastward_stress_SON_04, surface_downward_eastward_stress_SON_05,
                                                      surface_downward_eastward_stress_SON_06, surface_downward_eastward_stress_SON_07,
                                                      surface_downward_eastward_stress_SON_08, surface_downward_eastward_stress_SON_09,
                                                      surface_downward_eastward_stress_SON_10, surface_downward_eastward_stress_SON_11,
                                                      surface_downward_eastward_stress_SON_12, surface_downward_eastward_stress_SON_13,
                                                      surface_downward_eastward_stress_SON_14, surface_downward_eastward_stress_SON_15,
                                                      surface_downward_eastward_stress_SON_16, surface_downward_eastward_stress_SON_17,
                                                      surface_downward_eastward_stress_SON_18, surface_downward_eastward_stress_SON_19), axis = 0)

In [44]:
surface_downward_northward_stress_SON = np.concatenate((surface_downward_northward_stress_SON_93,
                                                       surface_downward_northward_stress_SON_94, surface_downward_northward_stress_SON_95,
                                                       surface_downward_northward_stress_SON_96, surface_downward_northward_stress_SON_97,
                                                       surface_downward_northward_stress_SON_98, surface_downward_northward_stress_SON_99,
                                                       surface_downward_northward_stress_SON_00, surface_downward_northward_stress_SON_01,
                                                       surface_downward_northward_stress_SON_02, surface_downward_northward_stress_SON_03,
                                                       surface_downward_northward_stress_SON_04, surface_downward_northward_stress_SON_05,
                                                       surface_downward_northward_stress_SON_06, surface_downward_northward_stress_SON_07,
                                                       surface_downward_northward_stress_SON_08, surface_downward_northward_stress_SON_09,
                                                       surface_downward_northward_stress_SON_10, surface_downward_northward_stress_SON_11,
                                                       surface_downward_northward_stress_SON_12, surface_downward_northward_stress_SON_13,
                                                       surface_downward_northward_stress_SON_14, surface_downward_northward_stress_SON_15,
                                                       surface_downward_northward_stress_SON_16, surface_downward_northward_stress_SON_17,
                                                       surface_downward_northward_stress_SON_18, surface_downward_northward_stress_SON_19), axis = 0)

In [45]:
np.savez(r'D:\heat_wave\WEIO\expand_WEIO\other_variables\last\wind_93_19_WEIO_area_SON.npz',
        wind_stress_SON = wind_stress_SON, surface_type_SON = surface_type_SON, eastward_wind_SON = eastward_wind_SON,
        wind_stress_curl_SON = wind_stress_curl_SON, northward_wind_SON = northward_wind_SON, 
        wind_speed_rms_SON = wind_speed_rms_SON, sampling_length_SON = sampling_length_SON,
        eastward_wind_rms_SON = eastward_wind_rms_SON, northward_wind_rms_SON = northward_wind_rms_SON,
        surface_downward_eastward_stress_SON = surface_downward_eastward_stress_SON,
        surface_downward_northward_stress_SON = surface_downward_northward_stress_SON)